# Facility Location Problem

Facility location problems, also known as location analysis [1], are a class of optimization problems that deal with finding the best locations for one or more facilities in a given space. The objective is usually to minimize the total transportation costs between the facilities and the demand points, such as customers, suppliers, or other facilities. However, there may be other factors to consider, such as environmental impact, market competition, or strategic goals.

Facility location problems have many practical applications in various domains. For example, in supply chain management and logistics, facility location problems can help us decide where to locate our stores, factories, warehouses, distribution centers, etc., to optimize the flow of goods and services. In public policy, facility location problems can help us plan the allocation of public resources, such as schools, hospitals, fire stations, police stations, etc., to serve the needs of the population. In telecommunications, facility location problems can help us design efficient and reliable networks, such as placing cell towers, routers, servers, etc., to maximize the coverage and quality of service. In particle physics, facility location problems can help us understand the behavior of subatomic particles, such as finding the optimal separation distance between repulsive charges. Another interesting application of facility location problems is cluster analysis, which is a technique for grouping similar data points based on their proximity.

In this tutorial, we will solve a specific facility location problem that arises in the retail industry using Pyomo. Suppose we have a set of supermarkets that need to be supplied by a set of warehouses. How many warehouses do we need, and where should we locate them, to minimize the total transportation costs between the warehouses and the supermarkets? To answer this question, we will formulate a mixed-integer programming (MIP) model of this problem, and use an example from the [Gurobi GitHub repository](https://github.com/Gurobi/modeling-examples/).

## Problem description

A supermarket chain in the UK needs to build warehouses for a set of supermarkets it is opening in Northern England. The locations of the supermarkets have been decided, but the locations of the warehouses have yet to be determined.

Several good candidate locations for the warehouses have been identified, but decisions must be made regarding how many warehouses to open and at which candidate locations to build them.

Opening many warehouses would be advantageous as this would reduce the average distance a truck has to drive from the warehouse to the supermarket, and hence reduce the delivery cost. However, opening a warehouse has a fixed cost associated with it.

In this example, our goal is to find the optimal tradeoff between delivery cost and the cost of building new facilities.

## Mathematical formulation

### Sets and parameters

Let us define the following sets and parameters for our problem:

- $I$: Set of supermarkets (customers) that we need to supply.
- $J$: Set of potential sites for the warehouses (facilities).

- $f_{j}$: Fixed cost for opening a facility at location $j \in J$.
- $d_{i,j}$: Distance between supermarket $i \in I$ and facility site $j \in J$.
- $c_{i,j}$: Transportation cost per unit of goods shipped from warehouse site $j \in J$ to supermarket $i \in I$. We assume that this cost is linearly proportional to the distance $d_{ij}$. That is, $c_{ij} = \alpha \, d_{ij}$, where $\alpha$ is a constant that represents the cost per mile of driving, adjusted for the expected number of trips a delivery truck would make over a five-year period.

### Decision variables

We use the following variables to represent our decisions:

- $y_{j}$: A binary variable that indicates whether we build a warehouse at site $j \in J$ or not. It is equal to 1 if we build a warehouse there, and 0 otherwise.

- $x_{i,j}$: A continuous variable that represents the fraction of goods that supermarket $i \in I$ receives from warehouse site $j \in J$. It is bounded between 0 and 1, meaning that each supermarket can receive goods from one or more warehouses to meet its demand.

### Objective function

Our goal is to minimize the total cost of building and running the warehouses. This cost consists of two parts: the facility cost and the delivery cost. The facility cost is the sum of the fixed costs of opening the warehouses at the chosen sites. The delivery cost is the sum of the transportation costs of shipping goods from the warehouses to the supermarkets. We want to find the best tradeoff between these two costs over a five-year period.

The objective function is:
$$
\min \ Z = \sum_{j \in J} f_{j}\, y_{j} + \sum_{i \in I} \sum_{j \in J} c_{ij}\, x_{ij}
$$

### Constraints

We need to satisfy the following constraints for our problem:

- Demand. Each supermarket must receive enough goods to meet its demand. This means that the sum of the fractions of goods that it receives from all the warehouses must be equal to 1:

$$
\sum_{j \in J} x_{ij} = 1, \quad \forall i \in I
$$

- Shipping. We can only ship goods from a warehouse site if we have built a warehouse there. This means that the fraction of goods that a supermarket receives from a warehouse site cannot exceed the binary variable that indicates whether we have built a warehouse there or not:

$$
x_{ij} \leq y_{j}, \quad \forall i \in I, \ j \in J
$$

## A problem instance

Let us consider a concrete example of our problem. We have two supermarkets and nine potential warehouse sites in a two-dimensional space. The following table gives the coordinates of the supermarkets.

|  | Coordinates |  
| --- | --- | 
| S1 | (0,1.5) |
| S2 | (2.5,1.2) |

The next table gives the coordinates and the fixed costs of the warehouse sites. The costs are in millions of GBP.

|  | Coordinates | Fixed cost |
| --- | --- |  --- |
| W1 | (0,0) | 3 |
| W2 | (0,1) | 2 |
| W3 | (0,2) | 3 |
| W4 | (1,0) | 1 |
| W5 | (1,1) | 3 | 
| W6 | (1,2) | 3 |
| W7 | (2,0) | 4 |
| W8 | (2,1) | 3 |  
| W9 | (2,2) | 2 |

We assume that the transportation cost per unit of goods shipped is one million GBP per mile. That is, $\alpha = 1$ in our model.

In [ ]:
# sets
I = set(["S1", "S2"])
J = set([f"W{i}" for i in range(1, 10)])

# parameters
location = {
    "S1": (0, 1.5),
    "S2": (2.5, 1.2),
    "W1": (0, 0),
    "W2": (0, 1),
    "W3": (0, 2),
    "W4": (1, 0),
    "W5": (1, 1),
    "W6": (1, 2),
    "W7": (2, 0),
    "W8": (2, 1),
    "W9": (2, 2),
}

fixed_cost = {"W1": 3, "W2": 2, "W3": 3, "W4": 1, "W5": 3, "W6": 3, "W7": 4, "W8": 3, "W9": 2}

alpha = 1.0

In [ ]:
# Visualize the points on the plain
from matplotlib import pyplot as plt

plt.scatter(
    x=[location[i][0] for i in location if i in I],
    y=[location[i][1] for i in location if i in I],
    c="red",
    label="I",
)
plt.scatter(
    x=[location[j][0] for j in location if j in J],
    y=[location[j][1] for j in location if j in J],
    c="blue",
    label="J",
    marker="s",
)

plt.legend()
plt.show()

Now, let's define a function to compute the shipping costs

In [ ]:
from math import sqrt


def shipping_cost(source, destination):
    x1, y1 = source
    x2, y2 = destination

    distance = sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    cost = alpha * distance

    return cost

## The concrete Pyomo model

In [ ]:
!pip install gurobipy pyomo

import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Include your WSL license information
solver_options = {}

In [ ]:
# Create the model
mod = pyo.ConcreteModel(name="FLP")

# Decision variables
mod.y = pyo.Var(J, domain=pyo.Binary)
mod.x = pyo.Var(I, J, domain=pyo.NonNegativeReals, bounds=(0, 1))

# Objective
expr = sum(fixed_cost[j] * mod.y[j] for j in J)
expr += sum(shipping_cost(location[i], location[j]) * mod.x[i, j] for i in I for j in J)
mod.obj = pyo.Objective(expr=expr, sense=pyo.minimize)

mod.obj.pprint()

In [ ]:
# Constraints
mod.demand = pyo.ConstraintList()
mod.shipping = pyo.ConstraintList()

for i in I:
    mod.demand.add(sum(mod.x[i, j] for j in J) == 1)

for i in I:
    for j in J:
        mod.shipping.add(mod.x[i, j] <= mod.y[j])

In [ ]:
# Call the solve and solve the model
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, solver_options=solver_options)
results = opt.solve(mod, tee=True)

## Solution analysis

- Objective value:

In [ ]:
print("Total cost: {:.4f} Million GBP".format(pyo.value(mod.obj)))

- Warehouse build plan: This plan determines the site locations at which we open a warehouse.

In [ ]:
warehouses = [j for j in J if mod.y[j].value > 0.5]
print("Opened warehouses:", warehouses)

- Shipment plan. This plan indicates how much goods each warehouse sends to each supermarket.

In [ ]:
for i in I:
    for j in J:
        val = 100 * pyo.value(mod.x[i, j])
        if val > 1.0e-6:
            print(f"Supermarket {i} receives {val:.1f}% of demand from warehouse {j}")

In [ ]:
# Visualize the assignments
from matplotlib import pyplot as plt

plt.scatter(
    x=[location[i][0] for i in location if i in I],
    y=[location[i][1] for i in location if i in I],
    c="red",
)
plt.scatter(
    x=[location[j][0] for j in location if j in J],
    y=[location[j][1] for j in location if j in J],
    c="blue",
    marker="s",
)

for i in I:
    for j in J:
        if pyo.value(mod.x[i, j]) > 1.0e-6:
            plt.plot(
                [location[i][0], location[j][0]],
                [location[i][1], location[j][1]],
                c="black",
                alpha=0.5,
            )

plt.show()

##  References

[1] Laporte, Gilbert, Stefan Nickel, and Saldanha da Gama, Francisco. Location Science. Springer, 2015.